In [2]:
import pandas as pd

In [3]:
# load the dataset
data = pd.read_csv('feature_seln_new.csv', parse_dates=['START_TIME'], index_col='START_TIME')
df=data[data.columns[1:]]

In [10]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [20, 5]

'''lamda_list = [40,41,42,43,44,45,46,47] 
window_size_list = [384,408,432,456,480,504,528,552]'''
lamda_list = [40,41,42,43,44,45,46,47] 
window_size_list = [24,168,672,1344]
no_col = len(df.columns)
no_rows = df.shape[0]

anomalies_array = np.zeros((len(lamda_list), len(window_size_list)))
#two_array = np.full((len(lamda_list), len(window_size_list)), 2)

for i in range(0,len(lamda_list)):
    lamda=lamda_list[i]
    for j in range(0,len(window_size_list)):
        window_size=window_size_list[j]
        col_anomalies=0
        for col in df.columns:
            rolling_mean = df[col].rolling(window=window_size).mean()
            std = np.std(rolling_mean)
            threshold = lamda * std
            '''plt.plot(df.index, df[col], label='Original Data')
            plt.plot(rolling_mean.index, rolling_mean, label='Moving Average')
            plt.legend(loc='best')'''
            upper_limit = rolling_mean + threshold
            lower_limit = rolling_mean - threshold
            anomalies = (df[col].abs() > upper_limit) | (df[col].abs() < lower_limit)
            num_anomalies = anomalies.sum()
            col_anomalies=col_anomalies+num_anomalies
            '''top_n_outliers = df.loc[anomalies, col].sort_values(ascending=False).head(top)
            plt.scatter(top_n_outliers.index, top_n_outliers.values, color='red', label='Anomalies')
            plt.title(f"{col}, lambda={lamda}, window_size={window_size}")
            plt.xlabel('Date')
            plt.ylabel('Value')
            #col_name = col.replace("/", " divided by ")
            #plt.savefig(f"{col_name}_moving_avg_lamda_{lamda}_window_{window_size}.png")
            plt.show()'''
        avg_anomalies=col_anomalies/no_col
        anomalies_array[i][j] = (avg_anomalies*100)/no_rows
        
        


In [11]:
print(anomalies_array)

[[4.96315024 4.94910437 4.44097429 3.6114416 ]
 [4.93918964 4.93753718 4.41701368 3.58995968]
 [4.93588472 4.9292749  4.40379404 3.56517285]
 [4.92844867 4.92266508 4.38809571 3.54121224]
 [4.92514376 4.91522903 4.37818098 3.52303523]
 [4.92101262 4.90614052 4.3699187  3.49907462]
 [4.91688149 4.8995307  4.35669905 3.48420252]
 [4.91522903 4.89457334 4.349263   3.47676647]]
